In [1]:
!pip install pandas

You should consider upgrading via the '/home/drapaiton/.cache/pypoetry/virtualenvs/tiendapago-examen-ea-xYsG0OIB-py3.9/bin/python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from pandas import DataFrame as DaFe

CUSTOMERS_FILE_PATH = "Customers.csv"
TRANSACTIONS_FILE_PATH = 'Transactions.csv'
DISTRIBUTOR_FILE_PATH = 'Distributor.csv'

TRANSACTION_AMOUNT_COLUMN = 'TransactionAmount'
DISTRIBUTOR_ID_COLUMN = 'DistributionCenterID'
CUSTOMER_ID_COLUMN = 'CustomerID'
DATE_COLUMN = 'Date'

DATE_FORMAT = '%m/%d/%Y %H:%M'

In [529]:
"""
1. Crear procesos de integración de datos que extraigan de los archivos insumos y generen
las transformaciones necesarias para un modelo de Business Intelligence y Data
Warehouse.

Empresa: Bytelian SA de CV
"""

def integrate_bi_input_files() -> DaFe:
    """generate a big DaFe to calculate easier, as this file grows strategy should be modified
    with chunk iteration, date iteration, unique client isolation  etc..."""
    try:
        customers_df = pd.read_csv(CUSTOMERS_FILE_PATH)
        transactions_df = pd.read_csv(TRANSACTIONS_FILE_PATH)
        distributor_df = pd.read_csv(DISTRIBUTOR_FILE_PATH)

        df = customers_df.join(transactions_df.set_index(CUSTOMER_ID_COLUMN), on=CUSTOMER_ID_COLUMN)
        if df.empty:
            FILES_TRIED_TO_JOIN = [CUSTOMERS_FILE_PATH,TRANSACTIONS_FILE_PATH]
            raise ValueError(f"these files couldn't be merged {FILES_TRIED_TO_JOIN}")

        df = df.join(distributor_df.set_index(DISTRIBUTOR_ID_COLUMN), on=DISTRIBUTOR_ID_COLUMN)
        if df.empty:
            FILES_TRIED_TO_JOIN = [[CUSTOMERS_FILE_PATH,TRANSACTIONS_FILE_PATH],DISTRIBUTOR_FILE_PATH]
            raise ValueError(f"these files couldn't be merged {FILES_TRIED_TO_JOIN}")

        # clean empty indexes (this should filter wrong data written at original file)
        df[CUSTOMER_ID_COLUMN] = df[CUSTOMER_ID_COLUMN].dropna()
        df[DISTRIBUTOR_ID_COLUMN] = df[DISTRIBUTOR_ID_COLUMN].dropna()

        # date parse
        df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN], format=DATE_FORMAT, errors='ignore')
        if df.empty:
            raise ValueError(f"couldn't parse {DATE_COLUMN=}")
    except Exception as e:
        raise e
    else:
        return df

def dag_bytelian_transactions_pipeline():
    """generate a big DaFe to calculate easier, as this file grows strategy should be modified
    with chunk iteration, date iteration, unique client isolation  etc..."""
    return integrate_bi_input_files()

dag_bytelian_transactions_pipeline()\
    .to_csv('pipeline_results.csv')

In [476]:
"""
2. Previo a un análisis de los datos, crear un modelo de Business Intelligence
a nivel analítico para el área comercial.
"""

full_df = dag_bytelian_transactions_pipeline()

def customers_with_higher_transaction_amount_sum(my_df:DaFe) -> dict:
    ACC_COLUMN_NAME = "Monto Colocado"
    # get sum of groups
    df = my_df[[CUSTOMER_ID_COLUMN, TRANSACTION_AMOUNT_COLUMN]]\
    .groupby(CUSTOMER_ID_COLUMN)\
    .agg(["sum"])\
    .reset_index()\
    .set_axis([CUSTOMER_ID_COLUMN, ACC_COLUMN_NAME], axis="columns")
    # return higher result
    return df.sort_values(ACC_COLUMN_NAME,ascending=False).head(5).reset_index(drop=True)

customers_with_higher_transaction_amount_sum(full_df)

,CustomerID,Monto Colocado
0,430695,395480.00
1,104503,363058.90
2,186171,323326.44
3,438341,292277.25
4,427121,289441.58


In [527]:
"""
2. Previo a un análisis de los datos, crear un modelo de Business Intelligence
a nivel analítico para el área comercial.
"""

full_df = dag_bytelian_transactions_pipeline()
# i would rather prefer to iterate thru months,
# but as dataset size is micro, daily is more explanatory
def customer_with_higher_transaction_amount_sum_per_day(my_df: DaFe) -> DaFe:
    df = my_df.copy()
    df[DATE_COLUMN] = df[DATE_COLUMN].dt.date
    df_sums = (
        df[[DATE_COLUMN,CUSTOMER_ID_COLUMN,TRANSACTION_AMOUNT_COLUMN]]
        .groupby([DATE_COLUMN,CUSTOMER_ID_COLUMN],as_index=False)
        .sum()
    )
    df_max = df_sums.groupby(DATE_COLUMN)\
        .idxmax()\
        .reset_index()\
        .set_index(TRANSACTION_AMOUNT_COLUMN)\
        .drop(CUSTOMER_ID_COLUMN, axis=1)
    return df_sums.loc[df_max.index].reset_index(drop=True)

result = customer_with_higher_transaction_amount_sum_per_day(full_df)
print(result.describe()[[TRANSACTION_AMOUNT_COLUMN]].apply(round, args=[1]))
result

       TransactionAmount
count               29.0
mean             59984.8
std              28615.7
min               9994.0
25%              39800.0
50%              55152.0
75%              79899.7
max             138935.9


,Date,CustomerID,TransactionAmount
0,2019-07-01,196391,79900.00
1,2019-07-02,104503,138935.90
2,2019-07-03,429356,27300.00
3,2019-07-04,427121,49568.71
4,2019-07-05,432698,37476.07
5,2019-07-06,432698,41573.93
6,2019-07-08,368806,49249.00
7,2019-07-09,365337,39417.18
8,2019-07-10,430695,79900.00
9,2019-07-11,436373,31600.00


In [528]:
"""
3. Calcular 2 métricas (monto colocado y número transacciones) donde:
a. Monto Colocado es la suma de la transacción (transaction amount).
b. Número de transacciones es el conteo de las transacciones totales.
"""

def calculate_sum_count_metrics(
    df: DaFe,
    output_sum_column,
    output_count_column,
    parent_column=CUSTOMER_ID_COLUMN,
    children_column=TRANSACTION_AMOUNT_COLUMN,
):
    return (
        df[[parent_column, children_column]]
        .groupby(parent_column)
        .agg(["sum", "count"])
        .reset_index()
        .set_axis(
            [parent_column, output_sum_column, output_count_column], axis="columns"
        )
    )

full_df = dag_bytelian_transactions_pipeline()
calculate_sum_count_metrics(full_df, "Monto Colocado", "Número de transacciones")\
    .to_excel('two_metrics.xlsx')

In [ ]:
"""
4. Proponer y justificar 3 métricas, que creas son importantes para la toma de decisiones del
gerente Hugo Montoya.


"""

In [ ]:
"""
5. Generar un tablero de control que muestre los principales indicadores de una forma
amigable para que el gerente pueda tomar decisiones de una manera eficaz y sencilla en
diferentes puntos del tiempo.

NOTA: Nuestra empresa necesita de un tablero de control que muestre los
principales indicadores (5) para la correcta toma de decisiones del
gerente comercial Hugo Montoya.
"""


